
# 🧠 From Predictive to Prescriptive Churn Reduction with Gen AI

---

Retailers face increasing customer churn, rising acquisition costs, and fierce competition, making customer retention more critical than ever.
Despite investments in loyalty programs and CRM systems, many struggle with fragmented customer data, ineffective engagement strategies, and the inability to act in real-time—resulting in missed opportunities and declining customer lifetime value.

<div style="background: #f7f7f7; border-left: 5px solid #ff5f46; padding: 20px; margin: 20px 0; font-size: 18px;">
"A recent review of the State of Customer Churn revealed that the <b>average churn rate</b> for the entire business sector is <b>6.58%</b>, far higher than the <b>ideal rate of 3-5%</b>."[^3]
</div>

<div style="background: #f7f7f7; border-left: 5px solid green; padding: 20px; margin: 20px 0; font-size: 18px;">
"With <b>prescriptive analytics</b> in place, decision makers can quickly extract <b>actionable knowledge</b> from the available data and define <b>effective actions</b> for the future."[^2]
</div>

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/liza.png" style="width: 100px; vertical-align: middle; margin-right: 10px; float: left;" />
<br>
Liza, a Generative AI Engineer, is going to utilize Databricks AI in order to create a prescriptive system for her company to reduce customer churn. She will use the full power of her company's data platform to create tailored advertising to her at-risk customers to encourage new purchases.

[^1]: https://www.qualtrics.com/experience-management/customer/customer-churn/

[^2]: https://voziq.ai/featured/prescriptive-analytics-for-churn-reduction/

[^3]: https://www.custify.com/blog/customer-churn-guide/

[^4]: https://www.infobip.com/blog/how-generative-ai-can-help-reduce-churn

[^5]: https://www.salesforce.com/sales/analytics/customer-churn/

[^6]: https://graphite-note.com/customer-churn-prevention-predictive-analytics/

[^7]: https://churnzero.com/blog/customer-success-statistics/

[^8]: https://www.b2brocket.ai/blog-posts/ai-powered-churn-prediction



## The Demo: Generative AI for Smarter Churn Reduction  
<!-- 
<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/refs/heads/main/images/retail/lakehouse-churn/lakehouse-retail-c360-agent-functions.png" 
     style="float: right; margin-top: 10px; width: 700px; display: block;" /> -->

Databricks Generative AI is redefining churn management by allowing organizations to shift from reactive retention to proactive customer engagement. Unlike traditional predictive models that simply flag at-risk customers, Databricks AI Agent Systems allow to use those predictive models in a high-value business context.  

In this demo, we show a simple use-case for Databricks to help reduce customer churn. We will create an AI agent using the following functions, defined in the subsequent cells:


1. **Customer Churn Prediction**: We will create a function that will allow our GenAI agent to determine if a customer is at risk of churn, based on the AutoML model we defined in **section 4** of this demo.
2. **Customer Order Lookup**: Then, our agent needs to be able to look up that at-risk-of-churn customer's most recent orders in order to create personalized outreach.
3. **Personalized Customer Outreach**: Finally, our agent will use this outreach function to piece together a personalized ad outreach based on the customer's demographics and purchase history to keep them engaged and a buying customer!

By leveraging Generative AI, retailers can move beyond predicting churn to actively **preventing it—turning potential losses into fruitful customer relationships**.   



<!-- Architecture doc: https://docs.google.com/presentation/d/1YzY7jj4GVXaEUjVPFPI74qgy00mVO6rWXHwv6iz0-lM/edit#slide=id.gf7b9ec9ce5_0_1087 -->


In [0]:
%pip install mlflow==2.20.1 databricks-vectorsearch==0.40 databricks-feature-engineering==0.8.0 databricks-sdk==0.40.0
dbutils.library.restartPython()

In [0]:
%run ../_resources/00-setup $reset_all_data=false

## Function 1: Customer Churn Prediction



In [0]:
spark.sql("DROP FUNCTION IF EXISTS churn_predictor")

spark.sql(f"""
          CREATE OR REPLACE FUNCTION churn_predictor(id STRING)
RETURNS STRING
LANGUAGE SQL
COMMENT 'This tool predicts if a customer is at risk of churning'
RETURN
(
    SELECT CASE 
             WHEN ai_query(
                    'dbdemos_customer_churn_endpoint', 
                    named_struct(
                        'user_id', user_id,
                        'canal', MAX(canal),
                        'country', MAX(country),
                        'gender', MAX(gender),
                        'age_group', MAX(age_group),
                        'order_count', MAX(order_count),
                        'total_amount', MAX(total_amount),
                        'total_item', MAX(total_item),
                        'last_transaction', MAX(last_transaction),
                        'platform', MAX(platform),
                        'event_count', MAX(event_count),
                        'session_count', MAX(session_count),
                        'days_since_creation', MAX(days_since_creation),
                        'days_since_last_activity', MAX(days_since_last_activity),
                        'days_last_event', MAX(days_last_event)
                    ),
                    'STRING'
                 ) = '0' THEN 'NOT AT RISK'
             ELSE 'AT RISK'
           END
    FROM {catalog}.{db}.churn_user_features
    WHERE user_id = id GROUP BY user_id
)""")

### Example:

In [0]:
%sql
SELECT churn_predictor(
    '2d17d7cd-38ae-440d-8485-34ce4f8f3b46'
) AS prediction


##Function 2: Customer Order Lookup

In [0]:
from pyspark.sql import functions as F

(spark.read.table("churn_orders").dropDuplicates(["order_id"])
 .withColumn("order_description", F.concat(F.lit("The user most recently executed a purchase transaction with id "), F.col("order_id"), F.lit(" with "), F.col("item_count"), F.lit(" item(s), worth a total dollar amount of $"), F.col("amount"), F.lit(" which was executed on "), F.date_format(F.col("creation_date"), "yyyy-MM-dd")))
 .select("user_id", "order_description", "creation_date")
 ).write.option("overwriteSchema", "true").mode("overwrite").saveAsTable("churn_orders_description")

display(spark.table("churn_orders_description"))

In [0]:
spark.sql("DROP TABLE IF EXISTS churn_orders_lookup")
spark.sql(f"""
  CREATE OR REPLACE FUNCTION churn_orders_lookup(id STRING)
  RETURNS STRING
  LANGUAGE SQL
  RETURN (
    SELECT order_description
    FROM {catalog}.{db}.churn_orders_description
    WHERE user_id = id
    ORDER BY creation_date DESC
    LIMIT 1
  )
""")

### Example:

In [0]:
%sql
SELECT churn_orders_lookup("f3d4a9f9-47c5-4fba-96e9-c2c55fb20676")

##Function 3: Personalized Customer Outreach

In [0]:
spark.sql(f"""
  CREATE OR REPLACE FUNCTION user_details_lookup (
    input_user_id STRING
    COMMENT 'User ID of the customer to be searched'
  ) 
  RETURNS TABLE(
    firstname STRING,
    user_id STRING,
    channel STRING,
    country STRING,
    gender INT,
    age_group INT,
    generation STRING,
    churn INT,
    days_since_last_activity INT,
    todays_date TIMESTAMP
  )
  COMMENT "This function returns the customer details for a given customer User ID. The return fields include First Name, User ID, Channel (e.g. Mobile App, Phone, or Web App/Browser ), Country of Residence, Gender, Age Group, Generation, Churn flag (1 for predicted churn, 0 otherwise), Days Since Last Activity, and the Current Date (the format is yyyy-MM-ddTHH:mm:ss.SSS). Use this function when a User ID is given." 
  RETURN (
    SELECT
      firstname,
      user_id,
      canal as channel,
      country,
      gender,
      age_group,
      CASE
        when age_group between 0
        and 2 then "Generation Alpha"
        when age_group between 3
        and 4 then "Generation Z"
        when age_group between 5
        and 6 then "Millennials"
        when age_group between 7
        and 8 then "Generation X"
        when age_group between 9
        and 10 then "Baby Boomers"
      END AS generation,
      churn,
      days_since_last_activity,
      getdate() as todays_date
    FROM 
      {catalog}.{db}.churn_prediction
    WHERE
      user_id = input_user_id
  )
""")

### Example:

In [0]:
%sql
SELECT * FROM user_details_lookup('f3d4a9f9-47c5-4fba-96e9-c2c55fb20676')

### Define the model prompt

In [0]:
prompt = """
    Your only role is to write copy for a contemporary apparel retailer. All your responses are a piece of copy, nothing further.\n
    You can use this tool to answer questions:\n
    - user_details: takes a customer User ID and returns the customer details for a given User ID. The return fields include First Name, User ID, Channel, Country, Gender, Age Group, Generation, Churn, days since last activity, and todays date (in this format: yyyy-MM-ddTHH:mm:ss.SSS). Use this function when User ID is given.\n
    These are the rules you must respect when writing copy:
    - Always use todays_date in conjunction with country of residence to determine the current season of the year. This is a MANDATORY step.\n
    - Start every message by greeting the customer using their first_name. Do not include a final greeting.\n
    - Write in the language of the customers country of residenct.\n
    - Use the customers local language for the entire copy (do not mix languages). Never explicitly mention their country of residence.\n
    - Write short copy (SMS or Push Notification style) for canal PHONE and MOBILE, and long copy (email style) for canal WEBAPP
    - Never mention a customers age or gender.\n
    - Adjust to the lingo of the customers generation. Never explicitly mention their generation.\n
    - Include weather references based on the current season of the year at their country of residence.\n
    - Adjust the length of the copy to their preferred communication channel. Never explicitly mention the channel.\n
    - Consider the time elapsed since the last event date when writing copy.\n
    - If churn is 1 and/or days since last activity is high, add urgency to the copy. Never explicitly mention churn status or days since last activity\n
    """

### Call the LLM with the model prompt + results of SQL function:

In [0]:
user_details_spain_webapp = spark.sql("SELECT * FROM user_details_lookup('f3d4a9f9-47c5-4fba-96e9-c2c55fb20676')").toPandas().to_string(index=False)
user_details_france_mobile = spark.sql("SELECT * FROM user_details_lookup('4734d799-1260-4d2e-b075-e8ada56fa902')").toPandas().to_string(index=False)
user_details_usa_phone = spark.sql("SELECT * FROM user_details_lookup('6d9ab038-f36d-4e91-81ea-746b34bf0fb3')").toPandas().to_string(index=False)

### Example: Spanish Customer

In [0]:
model_request = prompt + user_details_spain_webapp

In [0]:
model_response = spark.sql(
  f"""
  SELECT ai_query(
  'databricks-meta-llama-3-1-405b-instruct',
  '{model_request}'
  ) AS model_response
  """
)

In [0]:
display(model_response)

### Example: American Customer

In [0]:
model_request = prompt + user_details_usa_phone

In [0]:
model_response = spark.sql(
  f"""
  SELECT ai_query(
  'databricks-meta-llama-3-1-405b-instruct',
  '{model_request}'
  ) AS model_response
  """
)

In [0]:
display(model_response)

##Function 3: Image Generation with Shutterstock API

In [0]:
%sh 
pip install openai

In [0]:
from openai import OpenAI
import os

In [0]:
# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
# DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
# Alternatively in a Databricks notebook you can use this:
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints"
)

response = client.images.generate(
    model="databricks-shutterstock-imageai",
    prompt = model_response.collect()[0][0] + "IMPORTANT: IMAGES ONLY, NO TEXT!"
)

# print(response.data[0].image[0])

In [0]:
import io, base64
from PIL import Image

# Assuming base64_str is the string value without 'data:image/jpeg;base64,'
img = Image.open(io.BytesIO(base64.decodebytes(bytes(response.data[0].b64_json, "utf-8"))))

In [0]:
display(img)

## Next Steps

Proceed to notebook **05.2-Agent-Creation-Guide** in order to package the above functions into an implementable AI Agent with Databricks!